export cells as modules?



In [ ]:
var fs = require('fs');
var path = require('path');
var rimraf = require('rimraf');
var importer = require('../Core');
var mkdirpSync = importer.import('mkdirp');
var getExtension = importer.import('cell extension')

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var MODULE_OUTPUT = __dirname + '/.modules';
if(fs.existsSync(MODULE_OUTPUT)) {
    rimraf.sync(MODULE_OUTPUT);
}

function exportNotebook(notebook, project = __dirname + '/..') {
    const notebooks = typeof notebook === 'string' ? [notebook] : notebook;
    return notebooks.reduce((cells, notebook) => {
        const name = path.basename(notebook).replace(/\.ipynb/ig, '');
        const parent = path.resolve(path.dirname(notebook)).replace(path.resolve(project), '');
        mkdirpSync(path.join(MODULE_OUTPUT, parent, name));

        return importer.getCells(notebook, ['*', 'code', 'markdown', 'raw'])
            .reduce((results, cell, i) => {
                const extension = getExtension(cell, notebook);
                // TODO: get previous markdown and extract name that leads back to the current cell
                const cellPath = path.join(MODULE_OUTPUT, parent, name, 'cell-' + i + extension);
                fs.writeFileSync(cellPath, cell.source.join(''));
                results.push(cellPath);
                return results;
            }, cells);
    }, []);
}
module.exports = exportNotebook;


get cell extension?


In [ ]:

function getExtension(cell, notebook) {
    var extension;
    if(cell.cell_type === 'markdown') {
        extension = '.md';
    } else if(cell.cell_type === 'raw') {
        extension = '.txt';
    } else if(cell.language === 'javascript') {
        if((cell.source || [cell.code]).join('').match(/describe\s*\(/igm)) {
            extension = '.spec.js'
        } else {
            extension = '.js';
        }
    } else if(cell.language === 'powershell') {
        extension = '.ps1';
    } else if(cell.language === 'csharp') {
        extension = '.cs';
    } else if(cell.language === 'python') {
        extension = '.py';
    } else if(cell.language === 'typescript') {
        if((cell.source || [cell.code]).join('').match(/describe\s*\(/igm)) {
            extension = '.spec.ts'
        } else {
            extension = '.ts';
        }
    } else if(cell.language === 'bash') {
        extension = '.sh';
    } else {
        throw 'unknown language or cell type: ' + (cell.filename || notebook) + ' (' + cell.cell_type + ',' + cell.language + ')';
    }
    return extension;
}
module.exports = getExtension;


export all notebooks?

Export all notebooks to a structured folder where each cell has it's own file for linting?



In [ ]:
var glob = require('glob');
var path = require('path');
var importer = require('../Core');
var exportNotebook = importer.import('notebook.ipynb[export cells modules]');

function exportAll(project) {
    const notebooks = glob.sync('**/*.ipynb', {cwd: project});
    return exportNotebook(notebooks.map(n => path.join(project, n)), project);
}
module.exports = exportAll;


test notebook export?



In [ ]:
var path = require('path');
var glob = require('glob');
var assert = require('assert');
var fs = require('fs');
var importer = require('../Core');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = path.resolve(__dirname + '/../');
var MODULE_OUTPUT = PROJECT_PATH + '/Utilities/.modules';

var sortNumeric = (that) => {
    return that.sort((a, b) => parseInt(a.split(/[^0-9]/ig).join('')) - parseInt(b.split(/[^0-9]/ig).join('')));
}

describe('notebook export service', () => {
    it('should export this file', () => {
        exportNotebook(path.resolve(PROJECT_PATH + '/Utilities/notebook.ipynb'));
        var files = glob.sync('Utilities/notebook/*', {cwd: MODULE_OUTPUT});
        sortNumeric(files);
        assert(path.basename(files[0]) === 'cell-0.md');
        assert(path.basename(files[1]) === 'cell-1.js');
        assert(path.basename(files[2]) === 'cell-2.md');
    })
    
    it('should have comparable cells', () => {
        var importCells = importer.interpret(['notebook.ipynb[0]', 'notebook.ipynb[1]', 'notebook.ipynb[2]']);
        exportNotebook(path.resolve(PROJECT_PATH + '/Utilities/notebook.ipynb'));
        var files = glob.sync('Utilities/notebook/*', {cwd: MODULE_OUTPUT});
        sortNumeric(files);
        var fsCell0 = fs.readFileSync(path.join(MODULE_OUTPUT, files[0])).toString();
        var fsCell1 = fs.readFileSync(path.join(MODULE_OUTPUT, files[1])).toString();
        var fsCell2 = fs.readFileSync(path.join(MODULE_OUTPUT, files[2])).toString();
        assert(fsCell0 === importCells[0].markdown.join(''));
        assert(fsCell1 === importCells[0].fresher, fsCell1.length + ' != ' + importCells[0].fresher.length)
        assert(fsCell2 === importCells[1].markdown.join(''))
    })
    
    it('should export all notebooks', () => {
    //    var exported = exportAll(PROJECT_PATH);
    //    assert(exported.length > 0);
    })
})


import all cell modules?

import files back in to cells? (two-way workflow)


In [ ]:
var fs = require('fs');
var glob = require('glob');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Documents/jupytangular2/Utilities/.modules';
var project = PROFILE_PATH + '/Documents/jupytangular2';

var cells = glob.sync('**/cell-*', {cwd: PROJECT_PATH});
for(const c of cells) {
    const cell = path.basename(c);
    const notebook = path.basename(path.dirname(c));
    const parent = path.basename(path.dirname(path.dirname(c)));
    if(parent === 'jupytangular2') {
        continue;
    }
    const nb = JSON.parse(fs.readFileSync(path.join(project, parent, notebook + '.ipynb')));
    let counter = 0;
    for(const i in nb.cells) {
        if(!nb.cells.hasOwnProperty(i)) {
            continue;
        }
        // TODO: reimport all cells
        if(nb.cells[i].cell_type === 'code') {
            if(cell === 'cell-' + counter + '.js') {
                nb.cells[i].source = fs.readFileSync(path.join(PROJECT_PATH, c)).toString().split('\n');
            }
            counter++;
        }
    }
    fs.writeFileSync(path.join(project, parent, notebook + '.ipynb'), JSON.stringify(nb, null, 2));
}


In [ ]:
// TODO: delete .modules if everything checks out

// TODO: import tests


TODO:

import module as notebook cell?

import gist as notebook

import random instructions as notebook

rules:
- testing cells have zero markdown before them,
- all public functions are made public using module.export or global namespace
- use describe( test blocks to describe parsing and replacement information for the functions it is testing, i.e. function utility(root) would have a describe block describe('utility(root)') neatly formated and parsed for calendar commands
- every file is one feature
- every cell or export should be a single purpose component with proper includes/dependencies
- every cell must be under 100 lines
- every function must be testable insolation
- every function should be made accesible from the command line using module.exports
- every function should be runnable from notebooks using typeof $$ !== 'undefined'


```javascript

if(typeof $$ !== 'undefined') {
    $$.async();
    exportAndDeploy('../Frameworks/zuora to eloqua.ipynb')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}

```
- every module should have a markdown title, at least one question that the code intends to answer (how to?), includes at least (where name is the name of any function in the code block):


```javascript

...
function <name> () {
...
}
...
exports = <name>
...

```




get imports recursively?


In [ ]:
var importer = require('../Core');

function regexToArray(ex, str, i = 0) {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i])) ;
    return co;
};

function matchImports(code) {
    var imports = regexToArray(/(importNotebook|import)\(((\s*([\['"\{])\s*([\s\S]*?)\s*\3[,\s]*)+)\)/igm, code, 2);
    return imports.map(i => {
        const importStrings = regexToArray(/([\['"\{])\s*([\s\S]*?)\s*\1/igm, i, 2);
        if(i.indexOf('[') > -1) {
            return importStrings;
        }
        return importStrings[0];
    });
}

function getImportsRecursively(imports, cell) {
    if(typeof imports[cell.id] !== 'undefined') {
        return imports[cell.id];
    }
    cell.code = cell.code.replace(/require\('\.\.\/Core'\).*\s*/igm, 'importer.import(\'import notebook.ipynb\');\n');
    imports[cell.id] = matchImports(cell.code);
    imports[cell.id].forEach(arr => {
        if(typeof arr === 'string') {
            arr = [arr];
        }
        arr.forEach(i => {
            const result = importer.interpret(i);
            if(result.length) {
                result.forEach(r => {
                    getImportsRecursively(imports, r);
                });
            } else {
                getImportsRecursively(imports, result);
            }
        });
    });
    return imports[cell.id];
}
module.exports = getImportsRecursively;



replace notebook imports?


In [1]:
var _ = require('lodash');
var importer = require('../Core');
var {
    exportNotebook,
    getParameterNames
} = importer.import([
    'notebook.ipynb[export cells modules]',
    'rpc get result',
    'get function parameters'
]);

var REGEX_EXPORTS = /exports[^=]*?\s*=\s*([^ ]*)\s*;/igm;
var REGEX_NAMED = /function\s+([^\( ]*)\s*\(/igm

function regexToArray(ex, str, i = 0) {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i])) ;
    return co;
};

function getExports(code) {
    const named = regexToArray(REGEX_NAMED, code, 1);
    const namedExports = regexToArray(REGEX_EXPORTS, code, 1);
    return _.intersection(named, namedExports);
}

// TODO: get previous markdown and extract name that leads back to the current cell
function niceName(cell) {
    cell.questions.sort((a, b) => a.length - b.length);
    return cell.questions[0].replace('?', '').replace(/[^a-z0-9]+|\btest\b/igm, ' ').trim().replace(/\s+/igm, '-') + getExtension(cell);
}

function addImports(exportedCode, imports) {
    const newExports = getExports(exportedCode);
    // TODO: include all functional references from cells above this current cell
    Object.keys(imports).forEach(k => {
        const includedCell = importer.interpret(k);
        const funcName = getExports(includedCell.code)[0];
        if(funcName
            && newExports.indexOf(funcName) === -1
            && exportedCode.indexOf(funcName) > -1
            && !exportedCode.match(new RegExp('(let|var)\\s+' + funcName + '|(let|var)\\s+\\{[^\\}]*?' + funcName, 'igm'))
            && includedCell.questions.length > 0) {
            // TODO: replace this with a polyfill across ALL notebooks
            exportedCode = 'var ' + funcName + ' = (() => { const r = require(\'./' + niceName(includedCell) + '\'); return typeof r === \'function\' ? r : (typeof r === \'object\' && r !== null && typeof r[Object.keys(r)[0]] === \'function\' ? r[Object.keys(r)[0]] : r); })();\n' + exportedCode;
        }
    });
    return exportedCode;
}

function replaceObvious(code) {
    return code
        .replace(/require\('\.\.\/Core'\).*\s*/igm, 'importer.import(\'import notebook.ipynb\');\n')
        .replace(/.*typeof \$\$[\s\S]*?\}/igm, '')
//        .replace(/\bvar\b/igm, 'let');
}

function replaceImports(exports, cell, allImports) {
    cell.name = niceName(cell);
    exportNotebook(cell.filename);
    var newCell = replaceObvious(cell.code);
    exports.forEach(i => {
        const result = importer.interpret(i);
        if(result.length) {
            newCell = newCell.replace(
                new RegExp('(importNotebook|importer\.import)\\([\\[\'"]' + i + '[\'"\\,]][\s\S]*?\\)', 'igm'),
                    '[' + result
                .filter(r => r.questions.length > 0)
                .map(r => 'require(\'./' + (r.name = niceName(r)) + '\')')
                .join(', ') + `].reduce((acc, r, i) => {
    if(typeof r === 'function') {
        acc[r.name] = r;
    } else if (typeof r === 'object' && r !== null
        && typeof r[Object.keys(r)[0]] === 'function') {
        acc[r[Object.keys(r)[0]].name] = r[Object.keys(r)[0]];
    }
    acc[i] = r;
    return acc;
}, {})`.replace(/\n/igm, ''));
            return;
        } else if (result.questions.length === 0) {
            return;
        }
        newCell = newCell.replace(
            new RegExp('(importNotebook|importer\.import)\\([\\[\'"]' + i + '[\'"\\,]][\s\S]*?\\)', 'igm'),
                'require(\'./' + (result.name = niceName(result)) + '\')');
    });
    return addImports(newCell, allImports);
}
module.exports = replaceImports;


/Users/briancullinan/jupytangular2/Utilities


Not going to work file system.ipynb[(=\{((:[^{}]++|\{(1)\})++)\})]
Not going to work node simple-imap.ipynb[log in to Gmail using imap-simple/node IMAP]
Not going to work zuora to eloqua.ipynb[Create an access key for AWS: https://console.aws.amazon.com/iam/homenc2=h_m_sc#/security_credential]
Not going to work data collection.ipynb[https://www.maricopacountyattorney.org/CivicAlerts.aspxAID=400]
Not going to work github.ipynb[Diff all package.json to identify project/stack/popularity trends]
Not going to work What is Selenium.ipynb[[Click here to open](http://localhost:6080/vnc.htmlpassword=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true&reconnect=true) in a separate browser]
Not going to work file system.ipynb[(=\{((:[^{}]++|\{(1)\})++)\})]


[Function: replaceImports]

export deploy notebook coordinator?



In [1]:
// assuming you've already run `aws configure`
var fs = require('fs');
var os = require('os');
var path = require('path');
var importer = require('../Core');
var {
    execCmd,
    delint,
    getExtension,
    getImportsRecursively,
    replaceImports
} = importer.import([
    'spawn child process',
    'delinting notebooks',
    'cell extension',
    'get imports recursively',
    'replace notebook imports'
]);

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var PROJECT_PATH = path.join(__dirname, '../.output');
if(!fs.existsSync(PROJECT_PATH)) {
    fs.mkdirSync(PROJECT_PATH);
}

function exportAndDeploy(notebook) {
    // these are the top level cells starting the import tree
    const entryCells = importer.interpret(notebook);
    var imports = {};
    for(var k = 0; k < entryCells.length; k++) {
        getImportsRecursively(imports, entryCells[k]);
    }
    Object.keys(imports).forEach((e, i) => {
        const cell = importer.interpret(e);
        if(cell.questions.length === 0) {
            return;
        }
        let exportedCode = replaceImports(imports[e], cell, imports);
        console.log('emitting ' + cell.name);
        fs.writeFileSync(path.join(PROJECT_PATH, cell.name), exportedCode);
    });
    // TODO: zip and upload to AWS
    return Promise.resolve(imports);
    //return delint(PROJECT_PATH)
}
module.exports = exportAndDeploy;

if(typeof $$ !== 'undefined') {
    $$.async();
    exportAndDeploy('rpc.ipynb')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


/Users/briancullinan/jupytangular2/Utilities/file system.ipynb


Not going to work file system.ipynb[(=\{((:[^{}]++|\{(1)\})++)\})]
Not going to work node simple-imap.ipynb[log in to Gmail using imap-simple/node IMAP]
Not going to work zuora to eloqua.ipynb[Create an access key for AWS: https://console.aws.amazon.com/iam/homenc2=h_m_sc#/security_credential]
Not going to work data collection.ipynb[https://www.maricopacountyattorney.org/CivicAlerts.aspxAID=400]
Not going to work github.ipynb[Diff all package.json to identify project/stack/popularity trends]
Not going to work What is Selenium.ipynb[See [What is Docker](What%20is%20Docker.ipynb)]
Not going to work What is Selenium.ipynb[[Click here to open](http://localhost:6080/vnc.htmlpassword=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true&reconnect=true) in a separate browser]
Not going to work file system.ipynb[(=\{((:[^{}]++|\{(1)\})++)\})]


emitting rpc-permissions.js
emitting import-notebook.js
emitting stream-json-using-a-match-function.js
emitting parse-cells-of-types-from-a-notebook.js
emitting run-all-promises-sequentially.js
emitting search-notebook-questions.js
emitting find-all-notebooks.spec.js
emitting How-does-node-module-require-work.js
emitting Import-arbitrary-code-in-to-a-new-context.js
emitting read-markdown-leading-up-to-code-cells.js
emitting cache-notebook-questions-in-fuse.js
emitting display-interpreted-results-in-markdown.js
emitting turn-jupyter-notebooks-in-to-modules.js
emitting notebook-search.js
emitting filter-command-permission.js
emitting days-events.js
emitting Convert-a-date-to-ISO.js
emitting list-events.js
emitting use-Google-calendar-API.js
emitting authenticate-to-Google-APIs.js
emitting run-a-selenium-cell-on-the-Docker-machine.js
emitting start-a-webdriver-client-in-node.js
emitting find-webdriver-sessions.js
emitting Load-webdriver-sessions.js
emitting Manage-webdriver-sessions.js
em

{ 'rpc.ipynb[0]': [ 'import notebook.ipynb' ],
  'import notebook.ipynb[0]': [],
  'import notebook.ipynb[1]': [],
  'import notebook.ipynb[2]': [],
  'import notebook.ipynb[3]': [],
  'import notebook.ipynb[4]': [ 'interpret all notebooks.ipynb' ],
  'interpret all notebooks.ipynb[0]': [ 'import notebook.ipynb', 'interpret questions.ipynb' ],
  'import notebook.ipynb[5]': [],
  'import notebook.ipynb[6]': [],
  'interpret questions.ipynb[0]': [],
  'interpret questions.ipynb[1]': [],
  'interpret questions.ipynb[2]': [],
  'interpret questions.ipynb[3]': [ 'import notebook.ipynb' ],
  'interpret all notebooks.ipynb[1]': [],
  'rpc.ipynb[1]': 
   [ 'import notebook.ipynb',
     'rpc permissions',
     'days events',
     'google contact settings',
     'import google calendar api' ],
  'google calendar.ipynb[12]': 
   [ 'import notebook.ipynb',
     [ 'convert date iso', 'list events' ] ],
  'dates.ipynb[1]': [],
  'google calendar.ipynb[1]': 
   [ 'import notebook.ipynb',
     [ 'impo